In [1]:
import pandas as pd
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler, PowerTransformer
import string

stop_words = set(stopwords.words('german'))

[nltk_data] Downloading package stopwords to /Users/clara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/clara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/clara/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Extract the most important words from title, h1, abstract, url with NLP (e.g. TF-IDF )

In [2]:
df = pd.read_csv('/Users/clara/Desktop/neuefische/d-drivers/data/data_nlp.csv')

In [3]:
df.columns

Index(['page_id', 'n_days', 'url', 'n_urls', 'no_versions', 'age',
       'last_publish_date', 'publish_date_min', 'word_count',
       'classification_product', 'classification_type', 'page_name', 'title',
       'last_author', 'external_clicks', 'external_impressions',
       'likes_n_days', 'dislikes_n_days', 'video_play', 'page_impressions',
       'clickouts', 'ctr', 'h1', 'scraped_author', 'date_scraped', 'abstract',
       'scraped_word_count', 'meta_title', 'meta_description',
       'meta_image_url', 'media_type', 'page_img_size', 'merged_url',
       'meta_title_len', 'meta_desc_len', 'h1_len', 'abstract_len',
       'merged_url_len', 'sentiment_abstract', 'confidence_abstract',
       'sentiment_meta_title', 'confidence_meta_title'],
      dtype='object')

In [ ]:
# df_nlp = df[['page_id', 'external_clicks', 'external_impressions', 'ctr', 'h1', 
#         'abstract', 'meta_title', 'meta_description', 'merged_url']]

df_nlp = df

In [4]:
df_nlp.info()

NameError: name 'df_nlp' is not defined

In [ ]:
df_nlp.head()

,page_id,external_clicks,external_impressions,ctr,h1,abstract,meta_title,meta_description,merged_url
0,1037,256.0,5444.0,4.702425,Elektrofahrzeug-Ladestation: Kosten und Anbiet...,"Im Grunde kann man sein E-Auto überall laden, ...","Elektrofahrzeug-Ladestation: Anbieter, Kosten,...",Ladestationen für Elektroautos. Kosten und Anb...,"['elektrofahrzeug', 'ladestation']"
1,1039,124.0,1728.0,7.175926,Elektroauto-Förderung 2024: Wo Sie noch satt a...,Die große staatliche Förderung für Elektroauto...,E-Auto Förderung 2024: Alle Infos im Überblick,E-Auto Prämie 2024- so können Sie beim Kauf Ge...,"['absahnen', 'koennen', 'satt', '2024', 'sie',..."
2,1040,83.0,2575.0,3.223301,Hätten Sie es gewusst? So lange gibt es schon ...,Noch vor wenigen Jahren wurden Elektroautos al...,Hätten Sie es gewusst? So lange gibt es schon ...,Noch vor wenigen Jahren wurden Elektroautos al...,"['so', 'gewusst', 'e', 'haetten', 'autos', 'si..."
3,10245,80.0,845.0,9.467456,Kleinstes Auto der Welt kommt zurück: Fans kön...,"Der P.50, auch bekannt als das kleinste Auto d...",Kleinstes Auto der Welt kommt zurück: Fans kön...,"Der P.50, auch bekannt als das kleinste Auto d...","['zurueck', 'welt', 'fans', 'koennen', 'kommt'..."
4,10273,3384.0,94132.0,3.594952,Mehr Reichweite im Winter: So holen Sie alles ...,Wer freut sich schon über eisige Kälte? E-Auto...,Mehr Reichweite im Winter: So holen Sie alles ...,Wer freut sich schon über eisige Kälte? E-Auto...,"['so', 'e', 'ihrem', 'raus', 'reichweite', 'au..."


### Scale target variable

In [ ]:
scaler = PowerTransformer()
scaler.fit(df_nlp[['external_impressions']])
# Transform the target variable
df_nlp['external_impressions_scaled'] = scaler.transform(df_nlp[['external_impressions']])

/var/folders/5s/w7hst37d0tv0r4hby2_mqqp00000gn/T/ipykernel_44820/2008779672.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['external_impressions_scaled'] = scaler.transform(df_nlp[['external_impressions']])


### Remove stop words

In [ ]:
def remove_stopwords(text):
    if isinstance(text, str):        
        words = word_tokenize(text)
        # Remove punctuation and special characters
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove stopwords
        return ' '.join([word for word in words if word.lower() not in stop_words])
    else:
        return text

def remove_stopwords_from_columns(df, columns):
    for col in columns:
        df[col] = df[col].apply(remove_stopwords)
    return df

columns_to_clean = ['h1','abstract','meta_title','meta_description','merged_url']
df_nlp = remove_stopwords_from_columns(df_nlp, columns_to_clean)

/var/folders/5s/w7hst37d0tv0r4hby2_mqqp00000gn/T/ipykernel_44820/4158781637.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(remove_stopwords)
/var/folders/5s/w7hst37d0tv0r4hby2_mqqp00000gn/T/ipykernel_44820/4158781637.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(remove_stopwords)
/var/folders/5s/w7hst37d0tv0r4hby2_mqqp00000gn/T/ipykernel_44820/4158781637.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [ ]:
df_nlp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6815 entries, 0 to 6814
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   page_id                      6815 non-null   int64  
 1   external_clicks              6815 non-null   float64
 2   external_impressions         6815 non-null   float64
 3   ctr                          6815 non-null   float64
 4   h1                           6815 non-null   object 
 5   abstract                     6808 non-null   object 
 6   meta_title                   6815 non-null   object 
 7   meta_description             6815 non-null   object 
 8   merged_url                   6815 non-null   object 
 9   external_impressions_scaled  6815 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 532.6+ KB


In [ ]:
df_nlp.to_csv('/Users/clara/Desktop/neuefische/d-drivers/data/preprocessing_nlp.csv', encoding='utf-8', index=False)
